In [2]:
from sklearn import metrics
from datetime import datetime
from src.average_precision import calculate_score
import numpy as np
import pickle
from src.average_precision import calculate_delta
import pandas as pd
from src.average_precision import convert_to_names
from sklearn.model_selection import train_test_split, StratifiedKFold, GridSearchCV
from sklearn import metrics
import xgboost
from xgboost.sklearn import XGBClassifier
%matplotlib inline
import seaborn
from sklearn.externals import joblib

In [3]:
X = pd.read_pickle("data/multiclass_training.pickle")

In [4]:
label_encoder = pickle.load(open("models/label_encoder.pickle", "rb"))

In [5]:
cols_not_for_training = ["fecha_dato","ncodpers", "target", "product"]
X_train, X_test, y_train, y_test = train_test_split(
                                    X.drop(cols_not_for_training, axis=1), X["target"], test_size=0.20, random_state=42)

In [6]:
model = XGBClassifier(
        objective = 'multi:softprob',
        nthread = 4,
        learning_rate = 0.05,
        n_estimators = 100,
        max_depth = 8,
        subsample = 0.7,
        colsample_bytree = 0.7,
        colsample_bylevel = 0.7,
        silent = 0,
        )

param_grid = {}
param_grid["colsample_bytree"] = [0.3, 0.5, 0.6]
param_grid["colsample_bylevel"] = [0.3, 0.7, 1.]
param_grid["subsample"] = [0.7, 0.8, 1.]
param_grid["max_depth"] = [6, 8, 10]

grid_search = GridSearchCV(model, param_grid, scoring="accuracy", cv=3, verbose=3)
cv_results = grid_search.fit(X_train, y_train)


Fitting 3 folds for each of 81 candidates, totalling 243 fits
[CV] max_depth=6, subsample=0.7, colsample_bytree=0.3, colsample_bylevel=0.3 


/home/valentin/bin/anaconda3/lib/python3.5/site-packages/sklearn/model_selection/_split.py:581: Warning: The least populated class in y has only 2 members, which is too few. The minimum number of groups for any class cannot be less than n_splits=3.
  % (min_groups, self.n_splits)), Warning)


[CV]  max_depth=6, subsample=0.7, colsample_bytree=0.3, colsample_bylevel=0.3, score=0.531430, total=  19.4s
[CV] max_depth=6, subsample=0.7, colsample_bytree=0.3, colsample_bylevel=0.3 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   20.2s remaining:    0.0s


[CV]  max_depth=6, subsample=0.7, colsample_bytree=0.3, colsample_bylevel=0.3, score=0.527427, total=  20.5s
[CV] max_depth=6, subsample=0.7, colsample_bytree=0.3, colsample_bylevel=0.3 


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:   42.0s remaining:    0.0s


[CV]  max_depth=6, subsample=0.7, colsample_bytree=0.3, colsample_bylevel=0.3, score=0.539681, total=  42.3s
[CV] max_depth=6, subsample=0.8, colsample_bytree=0.3, colsample_bylevel=0.3 
[CV]  max_depth=6, subsample=0.8, colsample_bytree=0.3, colsample_bylevel=0.3, score=0.529624, total=  29.6s
[CV] max_depth=6, subsample=0.8, colsample_bytree=0.3, colsample_bylevel=0.3 
[CV]  max_depth=6, subsample=0.8, colsample_bytree=0.3, colsample_bylevel=0.3, score=0.526113, total=  25.5s
[CV] max_depth=6, subsample=0.8, colsample_bytree=0.3, colsample_bylevel=0.3 
[CV]  max_depth=6, subsample=0.8, colsample_bytree=0.3, colsample_bylevel=0.3, score=0.537134, total=  24.0s
[CV] max_depth=6, subsample=1.0, colsample_bytree=0.3, colsample_bylevel=0.3 
[CV]  max_depth=6, subsample=1.0, colsample_bytree=0.3, colsample_bylevel=0.3, score=0.529706, total=  25.7s
[CV] max_depth=6, subsample=1.0, colsample_bytree=0.3, colsample_bylevel=0.3 
[CV]  max_depth=6, subsample=1.0, colsample_bytree=0.3, colsample

KeyboardInterrupt: 

In [ ]:
joblib.dump(cv_results, "models/xgboost_cv.pickle")

In [10]:
preds = cv_results.predict(X_test)

In [20]:
print(metrics.classification_report(y_test, preds, target_names=list(label_encoder.classes_)))

                   precision    recall  f1-score   support

 ind_cco_fin_ult1       0.58      0.79      0.67      1921
ind_cder_fin_ult1       0.00      0.00      0.00         1
 ind_cno_fin_ult1       0.74      0.29      0.41       416
ind_ctju_fin_ult1       1.00      0.83      0.91        12
ind_ctma_fin_ult1       0.00      0.00      0.00        74
ind_ctop_fin_ult1       0.00      0.00      0.00        44
ind_ctpp_fin_ult1       0.00      0.00      0.00        30
ind_deco_fin_ult1       0.61      0.36      0.45        97
ind_dela_fin_ult1       0.78      0.08      0.15       224
ind_deme_fin_ult1       0.00      0.00      0.00         3
ind_ecue_fin_ult1       0.81      0.39      0.53       223
ind_fond_fin_ult1       0.00      0.00      0.00        50
 ind_hip_fin_ult1       0.47      0.64      0.54      1628
ind_nom_pens_ult1       0.11      0.01      0.02      1014
  ind_nomina_ult1       0.00      0.00      0.00         3
ind_plan_fin_ult1       0.00      0.00      0.00       

/home/valentin/bin/anaconda3/lib/python3.5/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [21]:
print(metrics.accuracy_score(y_test, preds))

0.547235905857
